In [1]:
import json
import random
from glob import glob

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-explain-sentiment/resolve/main/sentiment.jsonl

In [3]:
!head -n 1 sentiment.jsonl

{"sentiment": "positive", "explain_en": "The text is categorized as positive because it expresses gratitude or thankfulness.", "explain_ms": "Teks ini dikategorikan sebagai positif karena mengungkapkan rasa terima kasih atau syukur.", "text": "Terimakasih ya"}


In [17]:
english = [
    'bahasa inggeris standard',
    'standard english',
    'inggeris standard',
    'english'
]

malay = [
    'bahasa melayu standard',
    'standard malay',
    'melayu standard',
    'malay',
]

patterns = [
    'teks `{text}`, kelaskan sentimen [negative, positive, neutral] dan terangkan sebab dalam {lang}',
    'kelaskan sentimen [negative, positive, neutral] dan terangkan sebab dalam {lang}, teks `{text}`',
    'classify sentiment [negative, positive, neutral] and explain in {lang}, text `{text}`',
    'teks: {text}\nkelaskan sentimen [negative, positive, neutral] dan terangkan sebab dalam {lang}',
    'classify sentiment [negative, positive, neutral] and explain in {lang}\n\nteks: {text}'
]

patterns_json = [
    'teks `{text}`, kelaskan sentimen [negative, positive, neutral] dan terangkan sebab dalam {lang}, pulangkan dalam bentuk JSON {json}',
    'kelaskan sentimen [negative, positive, neutral] dan terangkan sebab dalam {lang}, bagi dalam bentuk JSON {json}, teks `{text}`',
    'classify sentiment [negative, positive, neutral] and explain in {lang}, text `{text}`, return in JSON {json}',
    'teks: {text}\nkelaskan sentimen [negative, positive, neutral] dan terangkan sebab dalam {lang}, bagi dalam JSON {json}',
    'classify sentiment [negative, positive, neutral] and explain in {lang}\n\nteks: {text}, return JSON {json}'
]


translation = []
with open('sentiment.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        q = None
        o = None
        try:

            while True:
                c = random.randint(0, 4)

                if c == 0 and l['explain_en']:
                    q = random.choice(patterns).format(text = l['text'], lang = random.choice(english))
                    o = f"Sentimen: {l['sentiment']}\nKeterangan: {l['explain_en']}"
                if c == 1 and l['explain_ms']:
                    q = random.choice(patterns).format(text = l['text'], lang = random.choice(malay))
                    o = f"Sentimen: {l['sentiment']}\nKeterangan: {l['explain_ms']}"
                if c == 2 and l['explain_en'] and l['explain_ms']:
                    lang = f'{random.choice(english)} dan {random.choice(malay)}'
                    q = random.choice(patterns).format(text = l['text'], lang = lang)
                    o = f"Sentimen: {l['sentiment']}\nKeterangan dalam Bahasa Melayu: {l['explain_ms']}\nKeterangan dalam Bahasa Inggeris: {l['explain_en']}"
                if c in [3, 4] and l['explain_en'] and l['explain_ms'] and l['sentiment']:
                    c = random.randint(0, 2)
                    if c == 0:
                        schema = "{'en', 'ms', 'text', 'sentiment'}"
                        d = {
                            'en': l['explain_en'],
                            'ms': l['explain_ms'],
                            'text': l['text'],
                            'sentiment': l['sentiment']
                        }
                    if c == 1:
                        schema = "{'explain_en', 'explain_ms', 'text', 'sentiment'}"
                        d = {
                            'explain_en': l['explain_en'],
                            'explain_ms': l['explain_ms'],
                            'text': l['text'],
                            'sentiment': l['sentiment']
                        }
                    if c == 2:
                        schema = "{'terang_en', 'terang_ms', 'teks', 'sentimen'}"
                        d = {
                            'terang_en': l['explain_en'],
                            'terang_ms': l['explain_ms'],
                            'teks': l['text'],
                            'sentimen': l['sentiment']
                        }
                    lang = f'{random.choice(english)} dan {random.choice(malay)}'
                    q = random.choice(patterns_json).format(text = l['text'], lang = lang, json = schema)
                    o = json.dumps(d)

                if q and o:
                    break

            translation.append({
                'prompt_input': None,
                'input': q.strip(),
                'output': o.strip(),
            })

        except Exception as e:
            print(e)

In [23]:
len(translation)

162902

In [25]:
translation[-4]

{'prompt_input': None,
 'input': "classify sentiment [negative, positive, neutral] and explain in standard english dan melayu standard, text `Semoga semuanya dipermudahkan. #engeartv #pandulalu #vaksin #labuan #covid19`, return in JSON {'en', 'ms', 'text', 'sentiment'}",
 'output': '{"en": "The text is expressing a positive sentiment. The author is hoping that everything will be made easier, possibly in relation to the COVID-19 situation in Labuan. The use of the hashtag #vaksin suggests that the author may be referring to the vaccination efforts in the area.", "ms": "Teks ini mengungkapkan sentimen positif. Penulis berharap bahwa segala sesuatunya akan dipermudah, mungkin terkait dengan situasi COVID-19 di Labuan. Penggunaan hashtag #vaksin menunjukkan bahwa penulis mungkin merujuk pada upaya vaksinasi di daerah tersebut.", "text": "Semoga semuanya dipermudahkan. #engeartv #pandulalu #vaksin #labuan #covid19", "sentiment": "positive"}'}

In [28]:
with open('prepared-synthetic-json-sentiment.jsonl', 'w') as fopen:
    for t in translation:
        fopen.write(f'{json.dumps(t)}\n')

In [29]:
!ls -lh prepared-synthetic-json-sentiment.jsonl

-rw-r--r-- 1 husein husein 103M Feb  24 12:26 prepared-synthetic-json-sentiment.jsonl
